In [1]:
#### Consolidate code from Geo Notebooks into a single block now that data is in the right format

### Import dataframe with geoghraphical of baseball fields

import pandas as pd
import geopy
from geopy import Point, distance


file_path = ('data/kml/ballparks.kml')


## Parse the kml file using the BeautifulSoup library
from bs4 import BeautifulSoup

## Initialize the BeautifulSoup object

with open(file_path) as f:
    xml_data = f.read()

soup = BeautifulSoup(xml_data, 'xml')

## Get the coordinates from the kml file
folders = soup.Document.Folder.find_all('Folder')
list = soup.Document.Folder.find_all('Folder')

### Create a dataframe with the coordinates of the baseball fields
df = pd.DataFrame(columns=['name', 'foul', 'fop'])

## Loop through the folders and extract the data


    
i = 0   

for i in range(len(list)):

    folders = list[i]
    field_name = folders.find('name').text
    foul = folders.find_all('coordinates')[0].text
    fop = folders.find_all('coordinates')[1].text

    row = {
        'field': field_name,
        'foul': foul,
        'fop': fop
    }

    i+=1

    df = df.append(row, ignore_index=True)



## Clean up the data
# remove new line and and space characters from coordinates
df = df.replace(r'\n','', regex=True) 
df = df.replace(r'\t','', regex=True) 


## Drop any duplicate rows
df = df.drop_duplicates(subset=['field'], keep='first')

## Drop and rows with empty fields
df = df[(df != 0).all(1)]

### Extract the home_plate coordinates by takeing the first coordinate in the foul coordinates column
df['home_plate'] = df['foul'].str.split(' ').str[0]

## drop the last 2 characters from the home_plate coordinates
df['home_plate'] = df['home_plate'].str[:-2]

### drop every ',0' from the fop coordinates
df['fop'] = df['fop'].str.replace(',0', '')

### reverse the order of the coordinates in the home_plate column
df['home_plate'] = df['home_plate'].str.split(',').str[::-1].str.join(',')

### Creaty a list of tuples for the fop coordinates
df['fop_list'] = df['fop'].apply(lambda x: [tuple(map(float, coord.split(','))) for coord in x.split()])

## Reverse the order of the values each coordinate tuple
df['fop_list'] = df['fop_list'].apply(lambda x: [tuple(reversed(coord)) for coord in x])



C:\Users\Justin\AppData\Local\Temp\ipykernel_516\369921160.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_516\369921160.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_516\369921160.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_516\369921160.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipyke

In [2]:
## Parse field names to get level column using regex
import re
re_mlb = re.compile(r'mlb', re.IGNORECASE)
re_pro = re.compile(r'pro', re.IGNORECASE)
re_college = re.compile(r'college', re.IGNORECASE)
re_youth = re.compile(r'youth', re.IGNORECASE)
re_muni = re.compile(r'muni', re.IGNORECASE)

df['level'] = df['field'].apply(lambda x: 'mlb' 
        if re_mlb.search(x) else 'pro' 
        if re_pro.search(x) else 'college' 
        if re_college.search(x) else 'youth' 
        if re_youth.search(x) else 'muni' 
        if re_muni.search(x) else 'high_school')

# clean up the field names
# remove the level from the field name
df['field'] = df['field'].str.replace(r'MLB', '')
df['field'] = df['field'].str.replace(r'pro', '')
df['field'] = df['field'].str.replace(r'college', '')
df['field'] = df['field'].str.replace(r'High School', 'HS')
# remove - from end of field name
df['field'] = df['field'].str.replace(r'- $', '')

## Output test csv
# df.to_csv('TEMP/level2_tost.csv', index=False)

C:\Users\Justin\AppData\Local\Temp\ipykernel_516\2064707252.py:23: FutureWarning: The default value of regex will change from True to False in a future version.
  df['field'] = df['field'].str.replace(r'- $', '')


In [3]:

### use the geopy distance method to get the distance between home_plate and each coordinate in the fop_list column for each row of the dataframe


### calculate the distance between home_plate and each coordinate in the fop_list column
df['distance'] = df.apply(lambda x: [distance.distance(x['home_plate'], coord).ft for coord in x['fop_list']], axis=1)

df['distance']

### Split the dataframe based on if the level is youth or not

youth_df = df[df['level'] == 'youth']
upper_df = df[df['level'] != 'youth']

# df.info()
upper_df.info()
youth_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 266 entries, 0 to 301
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        0 non-null      float64
 1   foul        266 non-null    object 
 2   fop         266 non-null    object 
 3   field       266 non-null    object 
 4   home_plate  266 non-null    object 
 5   fop_list    266 non-null    object 
 6   level       266 non-null    object 
 7   distance    266 non-null    object 
dtypes: float64(1), object(7)
memory usage: 18.7+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 17 to 285
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        0 non-null      float64
 1   foul        30 non-null     object 
 2   fop         30 non-null     object 
 3   field       30 non-null     object 
 4   home_plate  30 non-null     object 
 5   fop_list    30 non-null     object 
 

In [4]:
### Drop any distance values from the upper_df less than 200 feet and calculate the min and max distance
upper_df['distance'] = upper_df['distance'].apply(lambda x: [i for i in x if i > 180])
upper_df['min_distance'] = upper_df['distance'].apply(lambda x: min(x))
upper_df['max_distance'] = upper_df['distance'].apply(lambda x: max(x))
upper_df['avg_distance'] = upper_df['distance'].apply(lambda x: sum(x)/len(x))

## Drop any zero values from the youth_df and calculate the min and max distance

youth_df['distance'] = youth_df['distance'].apply(lambda x: [i for i in x if i > 100])
youth_df['min_distance'] = youth_df['distance'].apply(lambda x: min(x))
youth_df['max_distance'] = youth_df['distance'].apply(lambda x: max(x))
upper_df['avg_distance'] = youth_df['distance'].apply(lambda x: sum(x)/len(x))

# Recombine the dataframes
df = pd.concat([upper_df, youth_df])

### output the dataframe to a csv file

df.to_csv('data/viz/viz_book_distance.csv', index=False)


### FUCK YES IT WORKS!!!!

## some straNGE values, mostly in the min column. They all seem to be plots i did very early in the process.
## Those fields were probably maped differently than the process I eventually developed. I'll have to go back and fix them.



C:\Users\Justin\AppData\Local\Temp\ipykernel_516\3537169644.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upper_df['distance'] = upper_df['distance'].apply(lambda x: [i for i in x if i > 180])
C:\Users\Justin\AppData\Local\Temp\ipykernel_516\3537169644.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upper_df['min_distance'] = upper_df['distance'].apply(lambda x: min(x))
C:\Users\Justin\AppData\Local\Temp\ipykernel_516\3537169644.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [5]:
df['max_distance'].describe()

df['min_distance'].describe()

upper_df['distance'].describe()

count                                                   266
unique                                                  266
top       [322.9669999458172, 456.4168842027959, 448.645...
freq                                                      1
Name: distance, dtype: object

#### Code below has been modified to read a kml file that has been exported from Google My Maps and not yet been aggrigated in Google Earth


## First block of code takes the standard map stype with full foul and fop polygons


In [6]:


# ### Import dataframe with geoghraphical of baseball fields

# import pandas as pd
# import geopy
# from geopy import Point, distance


# file_path = ('data\kml\lansing_replots.kml')


# ## Parse the kml file using the BeautifulSoup library
# from bs4 import BeautifulSoup

# ## Initialize the BeautifulSoup object

# with open(file_path) as f:
#     xml_data = f.read()

# soup = BeautifulSoup(xml_data, 'xml')

# ## Get the coordinates from the kml file
# folders = soup.Document.find_all('Folder')
# list = soup.Document.find_all('Folder')

# ### Create a dataframe with the coordinates of the baseball fields
# df = pd.DataFrame(columns=['name', 'foul', 'fop'])

# ## Loop through the folders and extract the data


    
# i = 0   

# for i in range(len(list)):

#     folders = list[i]
#     field_name = folders.find('name').text
#     foul = folders.find_all('coordinates')[0].text
#     fop = folders.find_all('coordinates')[1].text
    

#     row = {
#         'field': field_name,
#         'foul': foul,
#         'fop': fop
    
#     }

#     i+=1

#     df = df.append(row, ignore_index=True)



# ## Clean up the data
# # remove new line and and space characters from coordinates
# df = df.replace(r'\n','', regex=True) 
# df = df.replace(r'\t','', regex=True) 


# ## Drop any duplicate rows
# df = df.drop_duplicates(subset=['field'], keep='first')

# ## Drop and rows with empty fields
# df = df[(df != 0).all(1)]

# ### Extract the home_plate coordinates by takeing the first coordinate in the foul coordinates column
# # df['home_plate'] = df['fop'].str.split(' ').str[0]

# ## drop the last 2 characters from the home_plate coordinates
# # df['home_plate'] = df['home_plate'].str[:-2]

# ### drop every ',0' from the fop coordinates
# df['fop'] = df['fop'].str.replace(',0', '')

# ### reverse the order of the coordinates in the home_plate column
# # df['home_plate'] = df['home_plate'].str.split(',').str[::-1].str.join(',')

# ### Creaty a list of tuples for the fop coordinates
# df['fop_list'] = df['fop'].apply(lambda x: [tuple(map(float, coord.split(','))) for coord in x.split()])

# ## Reverse the order of the values each coordinate tuple
# df['fop_list'] = df['fop_list'].apply(lambda x: [tuple(reversed(coord)) for coord in x])

# df['home_plate'] = df['fop_list'].apply(lambda x: x[0])

# ### use the geopy distance method to get the distance between home_plate and each coordinate in the fop_list column for each row of the dataframe


# ### calculate the distance between home_plate and each coordinate in the fop_list column
# df['distance'] = df.apply(lambda x: [distance.distance(x['home_plate'], coord).ft for coord in x['fop_list']], axis=1)

# df['distance']

# ### Drop any distances that are less than 200

# df['distance'] = df['distance'].apply(lambda x: [i for i in x if i <200])

# ## get the min from the distance column

# df['min_distance'] = df['distance'].apply(lambda x: min(x))

# ### get the max distance from home_plate to the fop coordinates
# df['max_distance'] = df['distance'].apply(lambda x: max(x))

# df.head()

# ### get the average distance from home_plate to the fop coordinates
# df['avg_distance'] = df['distance'].apply(lambda x: sum(x)/len(x))

# ### output the dataframe to a csv file

# df.to_csv('TEMP/college_tost.csv', index=False)

# # df.info()
# # df.head()